In [13]:
import os
import polars as pl

# Set environment variables to avoid using IMDS
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["AWS_REGION"] = "us-east-1"
os.environ["AWS_S3_USE_ARN_REGION"] = "true"
os.environ["AWS_ALLOW_HTTP"] = "true"

minio_storage_options = {
    "endpoint_url": "http://localhost:9000",
    "aws_access_key_id": "minio",
    "aws_secret_key_id": "minio123",
    "use_ssl": "false",
    "AWS_REGION": "us-east-1",
    "signature_version": "s3v4",
    # "AWS_ALLOW_HTTP": "true",
    # "AWS_S3_ALLOW_UNSAFE_RENAME": "true",
}

df = pl.DataFrame(
    {
        "foo": [1, 2, 3, 4, 5],
        "bar": [6, 7, 8, 9, 10],
        "ham": ["a", "b", "c", "d", "e"],
    }
)
import os 
os.makedirs("../../minio_data/depictio-bucket/TEST/", exist_ok=True)
df
df.write_delta(
    "../../minio_data/depictio-bucket/TEST/delta-table/", mode="overwrite", delta_write_options={"overwrite_schema": "True"}
    # storage_options=minio_storage_options,
)
import deltalake
df = deltalake.DeltaTable("../../minio_data/depictio-bucket/TEST/delta-table/")
deltalake.write_deltalake("../../minio_data/depictio-bucket/TEST/delta-table/", mode="overwrite", overwrite_schema=True, data=df.to_pandas())

os.environ["AWS_ALLOW_HTTP"] = "true"
os.environ["AWS_S3_ALLOW_UNSAFE_RENAME"] = "true"
deltalake.write_deltalake("s3://depictio-bucket/TEST/delta-table/", mode="overwrite", overwrite_schema=True, storage_options=minio_storage_options, data=df.to_pandas())
df.to_pandas()

# pl.scan_parquet(
#     "s3://depictio-bucket/TEST/delta-table.parquet",
#     storage_options={    "endpoint_url": "http://localhost:9000",
#     "aws_access_key_id": "admin",
#     "aws_secret_access_key": "alpiance"},
# )


/tmp/ipykernel_150322/3922160290.py:38: DeprecationWarning: overwrite_schema is deprecated, use schema_mode instead. 
  deltalake.write_deltalake("../../minio_data/depictio-bucket/TEST/delta-table/", mode="overwrite", overwrite_schema=True, data=df.to_pandas())


OSError: Generic S3 error: Client error with status 403 Forbidden: <?xml version="1.0" encoding="UTF-8"?>
<Error><Code>InvalidTokenId</Code><Message>The security token included in the request is invalid</Message><Key>TEST/delta-table/_delta_log/_last_checkpoint</Key><BucketName>depictio-bucket</BucketName><Resource>/depictio-bucket/TEST/delta-table/_delta_log/_last_checkpoint</Resource><RequestId>17D272C149DBA6AB</RequestId><HostId>dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8</HostId></Error>

In [6]:
from minio import Minio
from minio.error import S3Error

# Create a MinIO client
minio_client = Minio(
    "localhost:9000",
    access_key="minio",
    secret_key="minio123",
    secure=False
)

# Create a new bucket
bucket_name = "new-bucket"
try:
    minio_client.make_bucket(bucket_name)
    print(f"Bucket '{bucket_name}' created successfully.")
except S3Error as e:
    print(f"Error occurred: {e}")


Error occurred: S3 operation failed; code: BucketAlreadyOwnedByYou, message: Your previous request to create the named bucket succeeded and you already own it., resource: /new-bucket, request_id: 17D27375B0A363C3, host_id: dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8, bucket_name: new-bucket


In [21]:
# Add alias
!/workspaces/depictio/mc alias set myminio http://localhost:9000 minio minio123

# Add a new user
!/workspaces/depictio/mc admin user add myminio newuser newuser123


]11;?\Added `myminio` successfully.
]11;?\Added user `newuser` successfully.
]11;?\mc: <ERROR> Invalid command usage, flag provided but not defined: -user

SUPPORTED FLAGS:
   --config-dir value, -C value  path to configuration folder (default: "/home/vscode/.mc") [$MC_CONFIG_DIR]
   --quiet, -q                   disable progress bar display [$MC_QUIET]
   --disable-pager, --dp         disable mc internal pager and print to raw stdout [$MC_DISABLE_PAGER]
   --no-color                    disable color theme [$MC_NO_COLOR]
   --json                        enable JSON lines formatted output [$MC_JSON]
   --debug                       enable debug output [$MC_DEBUG]
   --insecure                    disable SSL certificate verification [$MC_INSECURE]
   --limit-upload value          limits uploads to a maximum rate in KiB/s, MiB/s, GiB/s. (default: unlimited) [$MC_LIMIT_UPLOAD]
   --limit-download value        limits downloads to a maximum rate in KiB/s, MiB/s, GiB/s. (default: unlimi

In [27]:
!pip install deltalake polars s3fs


In [10]:
from minio import Minio
from minio.error import S3Error
import io
import polars as pl
import deltalake

!unset AWS_ACCESS_KEY_ID
!unset AWS_SECRET_ACCESS_KEY
!unset AWS_SESSION_TOKEN
!unset AWS_SECURITY_TOKEN


# Create a new MinIO client with the new user's credentials
new_user_client = Minio(
    "localhost:9000",
    access_key="newuser",
    secret_key="newuser123",
    secure=False
)

# Verify write access by uploading a file
file_name = "testfile.txt"
content = b"Hello, this is a test file."
content_stream = io.BytesIO(content)

try:
    new_user_client.put_object(bucket_name, file_name, data=content_stream, length=len(content))
    print(f"File '{file_name}' uploaded successfully to bucket '{bucket_name}'.")
except S3Error as e:
    print(f"Error occurred during upload: {e}")

# Verify read access by downloading the file
try:
    response = new_user_client.get_object(bucket_name, file_name)
    data = response.read()
    print(f"File '{file_name}' downloaded successfully with content: {data.decode('utf-8')}")
except S3Error as e:
    print(f"Error occurred during download: {e}")

# Write a DeltaLake table using the new user's credentials
minio_storage_options = {
    "endpoint_url": "http://localhost:9000",
    "aws_access_key_id": "minio",
    "aws_secret_access_key": "minio123",
    "use_ssl": "False",
    "region_name": "us-east-1",
    "signature_version": "s3v4",
    "aws_s3_allow_unsafe_rename": "true",
    "aws_s3_use_arn_region": "true",
    "aws_allow_http": "true",
}

df = pl.DataFrame(
    {
        "foo": [1, 2, 3, 4, 5],
        "bar": [6, 7, 8, 9, 10],
        "ham": ["a", "b", "c", "d", "e"],
    }
)

try:
    deltalake.write_deltalake(f"s3://{bucket_name}/delta-table/", mode="overwrite", overwrite_schema=True, storage_options=minio_storage_options, data=df.to_pandas())
    print(f"Delta table written successfully to bucket '{bucket_name}'.")
except Exception as e:
    print(f"Error occurred while writing Delta table: {e}")


File 'testfile.txt' uploaded successfully to bucket 'new-bucket'.
File 'testfile.txt' downloaded successfully with content: Hello, this is a test file.
Error occurred while writing Delta table: Generic S3 error: Client error with status 403 Forbidden: <?xml version="1.0" encoding="UTF-8"?>
<Error><Code>InvalidTokenId</Code><Message>The security token included in the request is invalid</Message><Key>delta-table/_delta_log/_last_checkpoint</Key><BucketName>new-bucket</BucketName><Resource>/new-bucket/delta-table/_delta_log/_last_checkpoint</Resource><RequestId>17D273FF272218D4</RequestId><HostId>dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8</HostId></Error>


In [4]:
import logging
import boto3
import deltalake
import polars as pl



# Enable logging
boto3.set_stream_logger(name='botocore', level=logging.DEBUG)

minio_storage_options = {
    "endpoint_url": "http://localhost:9000",
    "aws_access_key_id": "cC1SHgv2W8XuG8izXXs1",
    "aws_secret_access_key": "xZEk76sQ4fZf5Qv9vUKcyRqStNjOoLFHRIRkcXJe",
    "use_ssl": "False",
    "region_name": "us-east-1",
    "signature_version": "s3v4",
    "aws_s3_allow_unsafe_rename": "true",
    "aws_s3_use_arn_region": "true",
    "aws_allow_http": "true",
}


df = pl.DataFrame(
    {
        "foo": [1, 2, 3, 4, 5],
        "bar": [6, 7, 8, 9, 10],
        "ham": ["a", "b", "c", "d", "e"],
    }
)

try:
    deltalake.write_deltalake(
        "s3://new-bucket/delta-table/",
        data=df.to_pandas(),
        mode="overwrite",
        storage_options=minio_storage_options
    )
    print("Delta table written successfully to S3.")
except Exception as e:
    print(f"Error occurred while writing Delta table: {e}")

Error occurred while writing Delta table: Generic S3 error: Client error with status 403 Forbidden: <?xml version="1.0" encoding="UTF-8"?>
<Error><Code>InvalidTokenId</Code><Message>The security token included in the request is invalid</Message><Key>delta-table/_delta_log/_last_checkpoint</Key><BucketName>new-bucket</BucketName><Resource>/new-bucket/delta-table/_delta_log/_last_checkpoint</Resource><RequestId>17D27366A541C4DE</RequestId><HostId>dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8</HostId></Error>
